I/O Performance (File Read/Write ) Code of all language

Pyhton - File Read/Write

In [ ]:
import pandas as pd
import time
import tracemalloc

# Start profiling
start_wall_time = time.time()
start_cpu_time = time.process_time()
tracemalloc.start()

# Step 1: Read the CSV file
input_file = r"C:\Users\c380b\OneDrive\Desktop\CDS\IO\IMDB Dataset.csv"
df = pd.read_csv(input_file)

# Step 2: Count the sentiment values
positive_count = (df['sentiment'].str.lower().str.strip() == 'positive').sum()
negative_count = (df['sentiment'].str.lower().str.strip() == 'negative').sum()

# Step 3: Prepare output data
output_text = f"Total positive reviews: {positive_count}\nTotal negative reviews: {negative_count}"

# Step 4: Write the result to a new file
output_file = r"C:\Users\c380b\OneDrive\Desktop\CDS\IO\sentiment_pyhton.txt"
with open(output_file, "w") as f:
    f.write(output_text)

# Stop profiling
end_wall_time = time.time()
end_cpu_time = time.process_time()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Print profiling results
print("Sentiment counts written to", output_file)
print(f"\n--- Profiling Report ---")
print(f"Wall-clock time   : {end_wall_time - start_wall_time:.4f} seconds")
print(f"CPU time          : {end_cpu_time - start_cpu_time:.4f} seconds")
print(f"Peak memory usage : {peak / 1024 / 1024:.2f} MB")   

Java - File Read/Write

In [ ]:
import java.io.*;
import java.lang.management.*;
import java.nio.file.*;
import java.util.*;

public class SentimentCounter {
    public static void main(String[] args) {
        // Set file paths
        String inputFile = "C:\\Users\\c380b\\OneDrive\\Desktop\\CDS\\IO\\IMDB Dataset.csv";
        String outputFile = "C:\\Users\\c380b\\OneDrive\\Desktop\\CDS\\IO\\sentiment_java.txt";

        int positiveCount = 0;
        int negativeCount = 0;

        // Start time and memory tracking
        long startWallTime = System.nanoTime();
        long startCpuTime = getCpuTime();
        // Force GC and measure initial memory
        System.gc();
        long startMemory = getUsedMemory();
        long startNonHeapMemory = getNonHeapMemory();

        try (BufferedReader reader = new BufferedReader(
                Files.newBufferedReader(Paths.get(inputFile)), 16 * 1024)) { // Increased buffer size
            String line;
            boolean firstLine = true;

            while ((line = reader.readLine()) != null) {
                if (firstLine) { // Skip header
                    firstLine = false;
                    continue;
                }

                // Minimize string operations
                if (line.endsWith(",positive")) {
                    positiveCount++;
                } else if (line.endsWith(",negative")) {
                    negativeCount++;
                }

                // Suggest GC periodically to manage temporary strings
                if ((positiveCount + negativeCount) % 10000 == 0) {
                    System.gc();
                }
            }

            // Write output efficiently
            List<String> outputLines = Arrays.asList(
                "Total positive reviews: " + positiveCount,
                "Total negative reviews: " + negativeCount
            );
            Files.write(Paths.get(outputFile), outputLines, StandardOpenOption.CREATE);
            System.out.println("Sentiment counts written to " + outputFile);

        } catch (IOException e) {
            System.err.println("Error reading or writing file: " + e.getMessage());
        }

        // End time and memory tracking
        // Force GC before final measurement
        System.gc();
        long endWallTime = System.nanoTime();
        long endCpuTime = getCpuTime();
        long endMemory = getUsedMemory();
        long endNonHeapMemory = getNonHeapMemory();

        // Display profiling report
        System.out.println("\n--- Profiling Report ---");
        System.out.printf("Wall-clock time       : %.4f seconds\n", (endWallTime - startWallTime) / 1e9);
        System.out.printf("CPU time              : %.4f seconds\n", (endCpuTime - startCpuTime) / 1e9);
        System.out.printf("Heap memory used      : %.2f MB\n", (endMemory - startMemory) / (1024.0 * 1024));
        System.out.printf("Non-heap memory used  : %.2f MB\n", (endNonHeapMemory - startNonHeapMemory) / (1024.0 * 1024));
        System.out.printf("Total memory used     : %.2f MB\n", 
            ((endMemory - startMemory) + (endNonHeapMemory - startNonHeapMemory)) / (1024.0 * 1024));
    }

    // Get CPU time using ManagementFactory
    private static long getCpuTime() {
        ThreadMXBean bean = ManagementFactory.getThreadMXBean();
        return bean.isCurrentThreadCpuTimeSupported() ? bean.getCurrentThreadCpuTime() : 0L;
    }

    // Get currently used heap memory
    private static long getUsedMemory() {
        Runtime runtime = Runtime.getRuntime();
        return runtime.totalMemory() - runtime.freeMemory();
    }

    // Get non-heap memory usage (e.g., metaspace)
    private static long getNonHeapMemory() {
        MemoryMXBean memoryBean = ManagementFactory.getMemoryMXBean();
        return memoryBean.getNonHeapMemoryUsage().getUsed();
    }
}

Go - File Read/Write

In [ ]:
package main

import (
	"bufio"
	"fmt"
	"os"
	"runtime"
	"runtime/pprof"
	"strings"
	"time"

	"github.com/shirou/gopsutil/v3/process"
)

// Warm-up to stabilize CPU and runtime
func warmUp() {
	for i := 0; i < 100000; i++ {
		_ = 3 * 3
	}
	runtime.GC()
	time.Sleep(100 * time.Millisecond)
}

// Process file and return counts
func processFile(inputFile string) (int, int, error) {
	file, err := os.Open(inputFile)
	if err != nil {
		return 0, 0, fmt.Errorf("error opening file: %v", err)
	}
	defer file.Close()

	var positiveCount, negativeCount int
	scanner := bufio.NewScanner(file)
	firstLine := true

	for scanner.Scan() {
		line := scanner.Text()
		if firstLine {
			firstLine = false
			continue
		}
		if strings.HasSuffix(line, ",positive") {
			positiveCount++
		} else if strings.HasSuffix(line, ",negative") {
			negativeCount++
		}
	}

	if err := scanner.Err(); err != nil {
		return 0, 0, fmt.Errorf("error reading file: %v", err)
	}
	return positiveCount, negativeCount, nil
}

// Poll memory usage
func measureMemory(p *process.Process, memChan chan float64, done chan struct{}) {
	for {
		select {
		case <-done:
			return
		default:
			var memStats runtime.MemStats
			runtime.ReadMemStats(&memStats)
			memMB := float64(memStats.Sys) / (1024.0 * 1024.0) // Use Sys for total memory
			select {
			case memChan <- memMB:
			default:
			}
			time.Sleep(10 * time.Millisecond)
		}
	}
}

func main() {
	// File paths
	inputFile := "C:\\Users\\c380b\\OneDrive\\Desktop\\CDS\\IO\\IMDB Dataset.csv"
	outputFile := "C:\\Users\\c380b\\OneDrive\\Desktop\\CDS\\IO\\sentiment_go.txt"
	cpuProfile := "C:\\Users\\c380b\\OneDrive\\Desktop\\CDS\\IO\\cpu.prof"
	const numRuns = 5 // Number of runs for averaging

	// Warm-up
	warmUp()

	// Create CPU profile file
	cpuFile, err := os.Create(cpuProfile)
	if err != nil {
		fmt.Println("Could not create CPU profile:", err)
		return
	}
	defer cpuFile.Close()

	// Initialize process for CPU time
	p, err := process.NewProcess(int32(os.Getpid()))
	if err != nil {
		fmt.Println("Error getting process info:", err)
		return
	}

	var totalWallTime, totalCPUTime float64
	var maxMemory float64
	var positiveCount, negativeCount int

	for run := 0; run < numRuns; run++ {
		// Start memory polling
		memChan := make(chan float64, 100)
		done := make(chan struct{})
		go measureMemory(p, memChan, done)

		// Start CPU profiling (only for first run to avoid overwriting)
		if run == 0 {
			pprof.StartCPUProfile(cpuFile)
		}

		// Start wall time and CPU time
		startWallTime := time.Now()
		cpuBefore, err := p.Times()
		if err != nil {
			fmt.Println("Error getting CPU time:", err)
			return
		}
		var memStart runtime.MemStats
		runtime.ReadMemStats(&memStart)
		memBefore := float64(memStart.Sys) / (1024.0 * 1024.0)

		// Process file
		positive, negative, err := processFile(inputFile)
		if err != nil {
			fmt.Println(err)
			return
		}
		if run == numRuns-1 {
			positiveCount = positive
			negativeCount = negative
		}

		// End measurements
		endWallTime := time.Now()
		cpuAfter, err := p.Times()
		if err != nil {
			fmt.Println("Error getting CPU time:", err)
			return
		}
		var memEnd runtime.MemStats
		runtime.ReadMemStats(&memEnd)
		memAfter := float64(memEnd.Sys) / (1024.0 * 1024.0)

		// Stop memory polling
		close(done)

		// Stop CPU profiling after first run
		if run == 0 {
			pprof.StopCPUProfile()
		}

		// Calculate times
		totalWallTime += endWallTime.Sub(startWallTime).Seconds()
		totalCPUTime += (cpuAfter.User + cpuAfter.System) - (cpuBefore.User + cpuBefore.System)

		// Update max memory
		localMaxMemory := memBefore
		if memAfter > localMaxMemory {
			localMaxMemory = memAfter
		}
		for len(memChan) > 0 {
			if mem := <-memChan; mem > localMaxMemory {
				localMaxMemory = mem
			}
		}
		if localMaxMemory > maxMemory {
			maxMemory = localMaxMemory
		}
	}

	// Write results to file (only once)
	out, err := os.Create(outputFile)
	if err != nil {
		fmt.Println("Error creating output file:", err)
		return
	}
	defer out.Close()
	fmt.Fprintf(out, "Total positive reviews: %d\n", positiveCount)
	fmt.Fprintf(out, "Total negative reviews: %d\n", negativeCount)
	fmt.Println("Sentiment counts written to", outputFile)

	// Calculate averages
	avgWallTime := totalWallTime / float64(numRuns)
	avgCPUTime := totalCPUTime / float64(numRuns)

	// Profiling Report
	fmt.Println("\n--- Profiling Report (Averaged over", numRuns, "runs) ---")
	fmt.Printf("Average Wall-clock time : %.4f seconds\n", avgWallTime)
	fmt.Printf("Average CPU time       : %.4f seconds\n", avgCPUTime)
	fmt.Printf("Peak memory usage      : %.2f MB\n", maxMemory)
	fmt.Printf("CPU profile saved      : %s\n", cpuProfile)
}


C++ - File Read/Write

In [ ]:
#include <iostream>
#include <fstream>
#include <string>
#include <chrono>
#include <windows.h>
#include <psapi.h>
#include <algorithm>
#include <cctype>

using namespace std;
using namespace std::chrono;

// Struct to hold memory info
struct MemoryUsage {
    size_t workingSetMB;
    size_t peakWorkingSetMB;
};

// Get current and peak memory usage
MemoryUsage getMemoryUsage() {
    PROCESS_MEMORY_COUNTERS memInfo;
    GetProcessMemoryInfo(GetCurrentProcess(), &memInfo, sizeof(memInfo));

    MemoryUsage usage;
    usage.workingSetMB = memInfo.WorkingSetSize / (1024 * 1024);
    usage.peakWorkingSetMB = memInfo.PeakWorkingSetSize / (1024 * 1024);
    return usage;
}

int main() {
    string inputFile = "C:\\Users\\c380b\\OneDrive\\Desktop\\CDS\\IO\\IMDB Dataset.csv";
    string outputFile = "C:\\Users\\c380b\\OneDrive\\Desktop\\CDS\\IO\\sentiment_cpp.txt";

    // Start profiling
    auto startWall = high_resolution_clock::now();
    clock_t cpuStart = clock();
    EmptyWorkingSet(GetCurrentProcess()); // Clear unused memory
    MemoryUsage memBefore = getMemoryUsage();

    ifstream inFile(inputFile);
    if (!inFile.is_open()) {
        cerr << "Failed to open file." << endl;
        return 1;
    }

    string line;
    int positiveCount = 0;
    int negativeCount = 0;

    // Skip header
    getline(inFile, line);

    while (getline(inFile, line)) {
        // Expecting format: "review","sentiment"
        size_t pos = line.rfind(',');
        if (pos != string::npos) {
            string sentiment = line.substr(pos + 1);

            // Clean up sentiment string
            sentiment.erase(remove(sentiment.begin(), sentiment.end(), '\"'), sentiment.end());
            sentiment.erase(remove_if(sentiment.begin(), sentiment.end(), ::isspace), sentiment.end());

            if (sentiment == "positive")
                ++positiveCount;
            else if (sentiment == "negative")
                ++negativeCount;
        }
    }

    inFile.close();

    // Write output
    ofstream outFile(outputFile);
    if (outFile.is_open()) {
        outFile << "Total positive reviews: " << positiveCount << "\n";
        outFile << "Total negative reviews: " << negativeCount << "\n";
        outFile.close();
    } else {
        cerr << "Failed to write to file." << endl;
        return 1;
    }

    // End profiling
    auto endWall = high_resolution_clock::now();
    clock_t cpuEnd = clock();
    EmptyWorkingSet(GetCurrentProcess());
    MemoryUsage memAfter = getMemoryUsage();

    // Calculate time and memory
    double wallTime = duration_cast<duration<double>>(endWall - startWall).count();
    double cpuTime = 1000.0 * (cpuEnd - cpuStart) / CLOCKS_PER_SEC;
    size_t usedMemoryMB = memAfter.workingSetMB > memBefore.workingSetMB
                          ? memAfter.workingSetMB - memBefore.workingSetMB
                          : 0;

    // Output report
    cout << "\nSentiment counts written to " << outputFile << endl;
    cout << "\n--- Profiling Report ---" << endl;
    cout << "Wall-clock time    : " << wallTime << " seconds" << endl;
    cout << "CPU time           : " << cpuTime << " ms" << endl;
    cout << "Used memory        : " << usedMemoryMB << " MB" << endl;
    cout << "Peak memory usage  : " << memAfter.peakWorkingSetMB << " MB" << endl;

    return 0;
}
